In [ ]:
%reload_ext autoreload

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import linalg
from quantum_tools.config import *
from quantum_tools.utilities import utils
from functools import reduce
from cmath import exp

In [3]:
def convex_param(θ):
    """
    Given a set of k parameters, find {p_n} such that Σn=1..k+1 p_n = 1
    """
    
    k = len(θ)
    p = np.zeros(k+1)
    
    cos2p = np.cos(θ)**2
    sin2p = np.sin(θ)**2
    
    for n in range(k):
        p[n] = cos2p[n] * np.prod(sin2p[:n])
    p[k] = np.prod(sin2p)
    
    return p

In [4]:
def is_unitary(U):
    return np.allclose(U.conj().T.dot(U), np.eye(U.shape[0]))

In [5]:
def get_Pl(d):
    """
    The imdepotent projector of P_l = |l><l| where |l> lives in H^d
    """
    Pl = np.zeros((d,d,d))
    for i in range(d):
        Pl[i,i,i] = 1
    return Pl

In [41]:
def expim(M, λ):
    return linalg.expm(1j * M * λ)

def pexpim(M, λ, I):
    return I + M *(exp(1j * λ) - 1)

# def sexpim(σ, m, n, λ, I):
#     return np.dot((I + σ[m,n] * λ), (I - σ[n,m] * λ))

def sexpim(p_σ, m, n, λ, I):
    return I + (p_σ[m,m] + p_σ[n,n]) * (np.cos(λ) - 1) + (p_σ[m,n] - p_σ[n,m]) * np.sin(λ)
#     return linalg.expm(σ )

In [8]:
def get_basis(d):
    return np.eye(d)

In [9]:
def get_σ_matrix(d):
    σ = []
    basis = get_basis(d)
    for m in range(d):
        σ_m = []
        for n in range(d):
            partial_σ_mn = 1j * np.outer(basis[n], basis[m])
            σ_mn = partial_σ_mn.conj().T + partial_σ_mn
            σ_m.append(σ_mn)
        σ.append(σ_m)
    return np.array(σ)

def get_partial_σ_matrix(d):
    p_σ = []
    basis = get_basis(d)
    for m in range(d):
        p_σ_m = []
        for n in range(d):
            p_σ_mn = np.outer(basis[m], basis[n])
            p_σ_m.append(p_σ_mn)
        p_σ.append(p_σ_m)
    return np.array(p_σ)

In [10]:
def multi_dot(Mv):
    return reduce(np.dot, Mv)

In [37]:
class UnitaryParam():
    
    def __init__(self, d):
        self.d = d
        self.basis = get_basis(d)
        self.I = np.eye(d)
        self.Pl = get_Pl(d)
#         self.σ = get_σ_matrix(d)
        self.p_σ = get_partial_σ_matrix(d)
        
    def GP(self, λ):
        return multi_dot(pexpim(self.Pl[l], λ[l,l], self.I) for l in range(self.d))
    
    def Λ(self, m, n, λ):
        return np.dot(pexpim(self.Pl[n], λ[n,m], self.I), sexpim(self.p_σ, m, n, λ[m,n], self.I))
    
    def U(self, λ):
        GP = self.GP(λ)
        RRP = multi_dot(multi_dot(self.Λ(m, n, λ) for n in range(m + 1, self.d)) for m in range(self.d-1))
        return np.dot(RRP, GP)

In [38]:
up = UnitaryParam(3)

In [56]:
print(up.U(λ))
print(up.U(λ).conj().T)
print(np.dot(up.U(λ).conj().T, up.U(λ)))

[[-0.28901-0.4501j  -0.72357-0.17289j -0.16165+0.36649j]
 [ 0.07548-0.01076j -0.20943-0.40166j  0.87138-0.17231j]
 [ 0.35018-0.76515j  0.19691+0.45011j  0.22203+0.03547j]]
[[-0.28901+0.4501j   0.07548+0.01076j  0.35018+0.76515j]
 [-0.72357+0.17289j -0.20943+0.40166j  0.19691-0.45011j]
 [-0.16165-0.36649j  0.87138+0.17231j  0.22203-0.03547j]]
[[ 1.+0.j -0.+0.j  0.+0.j]
 [-0.-0.j  1.+0.j  0.-0.j]
 [ 0.-0.j  0.+0.j  1.+0.j]]


In [55]:
m, n = 1,2
p_σ = get_partial_σ_matrix(3)
print(expim(-1j * p_σ[m,n] + 1j * p_σ[n,m], λ[m,n]))
print(sexpim(p_σ, m, n, λ[m,n], np.eye(3)))

[[ 1.00000+0.j  0.00000+0.j  0.00000+0.j]
 [ 0.00000+0.j -0.41615+0.j  0.90930+0.j]
 [ 0.00000-0.j -0.90930-0.j -0.41615-0.j]]
[[ 1.       0.       0.     ]
 [ 0.      -0.41615  0.9093 ]
 [ 0.      -0.9093  -0.41615]]


In [59]:
PROFILE_MIXIN(stress_test, 1000)

         65148 function calls (59148 primitive calls) in 0.198 seconds

   Ordered by: cumulative time
   List reduced from 25 to 5 due to restriction <0.2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.203    0.203 <ipython-input-50-ffe4f94e51db>:1(stress_test)
     1000    0.003    0.000    0.202    0.000 <ipython-input-37-439c1a8f12fc>:17(U)
5000/2000    0.003    0.000    0.195    0.000 <ipython-input-10-15722dcdcdde>:1(multi_dot)
5000/2000    0.019    0.000    0.194    0.000 {built-in method reduce}
     9000    0.004    0.000    0.135    0.000 <ipython-input-37-439c1a8f12fc>:19(<genexpr>)




In [53]:
4.00 / 0.06

66.66666666666667

In [307]:
λs = [np.random.normal(0,1,(d, d)) for _ in range(1000)]

In [50]:
def stress_test(k):
    d = 4
    up = UnitaryParam(4)
    λ = np.random.normal(0,1, (d, d))
    for _ in range(k):
        U = up.U(λ)
#         if not is_unitary(U):
#             print(U)
#             raise Exception("Ooops")

In [14]:
λ = np.array([
        [1,3,1],
        [2,2, 2],
        [1, 1.3, 1],
    ])

In [94]:
σ_matrix(0,3,basis(4))

array([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.-1.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+1.j,  0.+0.j,  0.+0.j,  0.+0.j]])

In [57]:
global_phase(3, λ)

NameError: name 'global_phase' is not defined

In [65]:
convex_param(np.array([np.pi/3, np.pi/4, np.pi/6]))

array([ 0.25   ,  0.375  ,  0.28125,  0.09375])

In [62]:
P_l(0, 4)

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])